# Lasenby Rotor Estimation

In [1]:
from __future__ import print_function

In [2]:
import sys
sys.path.append('../build/')

In [93]:
%pylab inline
np.set_printoptions(precision=4, suppress=True)

Populating the interactive namespace from numpy and matplotlib


In [94]:
import versor as vsr

## Generate Dataset

In [95]:
rotor = (vsr.Biv(1,1,1).unit() * pi/6.0  ).exp()
print(rotor)

Rot: [ 0.87 -0.29 -0.29 -0.29 ]


In [96]:
n_vectors = 10
sigma = 0.09
vectors_a = [vsr.Vec(*np.random.normal(0.0, 0.8, 3)).unit()
            for i in range(n_vectors)]
vectors_b = [vector.spin(rotor) for vector in vectors_a]
vectors_b_noisy = [vsr.Vec(*(np.array(vector)[:3] 
                            + sigma * np.random.randn(3))) 
                  for vector in vectors_b]

In [114]:
F = np.zeros((3,3))
for i in range(3):
    for j in range(3):
        sigma_i = vsr.Vec(0,0,0)
        sigma_j = vsr.Vec(0,0,0)
        sigma_i[i] = 1.0
        sigma_j[j] = 1.0
        F[i,j] = np.sum([(sigma_i <= ui) * (sigma_j <= vi) 
                         for ui, vi in zip(vectors_a, vectors_b)  ] )
U, s, Vt = np.linalg.svd(F)
R_lasenby = np.dot(Vt.T,U.T)
print(R_lasenby)

[[ 0.6667 -0.6667 -0.3333]
 [ 0.3333  0.6667 -0.6667]
 [ 0.6667  0.3333  0.6667]]


In [122]:
U, _, Vt = np.linalg.svd(np.sum([np.outer(np.array(ui), np.array(vi)) 
                                 for ui, vi in zip(vectors_a, vectors_b)], 0))
R_lasenby = np.dot(Vt.T,U.T)
print(R_lasenby)

[[ 0.6667 -0.6667 -0.3333]
 [ 0.3333  0.6667 -0.6667]
 [ 0.6667  0.3333  0.6667]]


In [123]:
R_real = np.array([np.array(vsr.Vec(1,0,0).spin(rotor)), 
                   np.array(vsr.Vec(0,1,0).spin(rotor)), 
                   np.array(vsr.Vec(0,0,1).spin(rotor))]).T

In [124]:
np.allclose(R_lasenby, R_real)

True